# Simulation of a quadruped Robot


Using the simulator the goal of this challenge is to come up with an Agent for controlling a Quadrupedal Robot on a flat environement. The best solution will be the solution that goes as far as possible given a number of Timesteps

## Using the Environment

You are provided with an environment that uses the [OpenAI Gym Interface](https://gym.openai.com/). 

Once initialized, you can interact with the environment using the step function that takes a set of actions ( a numpy array of size 12 between -1.0 and 1.0 that corresponds to target angles for each joints. ) 
The order of the joint angles is as follow [knee, hip, shoulder] for each of the legs in order [Front Left, Rear Right, Front Right, Rear Left]


In [1]:
import aida_env.aida_gym_env as e
import pybullet as p
import numpy as np


In [2]:
env = e.AidaBulletEnv(render=True, on_rack=False)  #  use on_rack to remove gravity


urdf_root=/Library/anaconda/lib/python2.7/site-packages/pybullet_data
aida_env/urdf/plane.urdf
/Library/anaconda/lib/python2.7/site-packages/pybullet_data/plane.urdf
(0.01, 1.0)
WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.


In [4]:
# Each joint moving one by one. 
class Agent():
    def get_actions(self, joint, v):
        actions = np.zeros(12)
        actions[joint] = v
        return actions

agent = Agent()

for i in range(200):
     env.step(np.zeros(12))  # init
for i in range(1200):
    joint = int(i / 100) # go through each joint one by one
    angle = (int(i / 50) % 2 - 1.0) # alternate between max and min angle for each joint. 
    env.render()
    env.step(agent.get_actions(joint, angle)) # take a random actio


TypeError: 'NoneType' object is not iterable

# Writing an Agent

You can write an Agent to control the actions sent to the environment. 

In [ ]:
#Close the windows and the env.
p.disconnect()


In [2]:
env = e.AidaBulletEnv(render=True, on_rack=False)  #  use on_rack to remove gravity


urdf_root=/Library/anaconda/lib/python2.7/site-packages/pybullet_data
aida_env/urdf/plane.urdf
/Library/anaconda/lib/python2.7/site-packages/pybullet_data/plane.urdf
(0.01, 1.0)
WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.


In [4]:
# A small baseline example.
class Agent():
    def __init__(self):
        self.t = 0
        
    def get_actions(self):
        self.t += 1
        if self.t < 200:
            return self.start()
        
        actions = self.circles(self.t)
        actions_unsync = self.circles(self.t ,offset = np.pi )
        actions = actions_unsync + actions_unsync + actions + actions
        return actions
    
    def start(self):
        return -0.5 * np.ones(12) # a good value to have the robot start on its legs

    def circles(self, t, offset=0, speed=0.06):
        return [-0.5 + np.sin(t * speed + offset) * 0.25, -0.5 + np.cos(t * speed + offset) * 0.25, -0.5]

agent = Agent()

In [ ]:
class Agent():
    def __init__(self, final_point):
        self.t = 0
        self.final_point = final_point
        
    def get_actions(self):
        self.t += 1
        if self.t < 200:
            return self.start()
        
        actions = self.circles(self.t)
        actions_unsync = self.circles(self.t ,offset = np.pi )
        actions = actions_unsync + actions_unsync + actions + actions
        return actions
    
    def start(self):
        return -0.5 * np.ones(12) # a good value to have the robot start on its legs

    def circles(self, t, offset=0, speed=0.06):
        return [-0.5 + np.sin(t * speed + offset) * 0.25, -0.5 + np.cos(t * speed + offset) * 0.25, -0.5]

agent = Agent([100,200])

In [5]:
for i in range(2000):
    observ = env._get_observation
    print(observ)
    env.step(agent.get_actions())
    #print(env.aida.GetBasePosition())

<bound method AidaBulletEnv._get_observation of <aida_env.aida_gym_env.AidaBulletEnv object at 0x102b10350>>
<bound method AidaBulletEnv._get_observation of <aida_env.aida_gym_env.AidaBulletEnv object at 0x102b10350>>
<bound method AidaBulletEnv._get_observation of <aida_env.aida_gym_env.AidaBulletEnv object at 0x102b10350>>
<bound method AidaBulletEnv._get_observation of <aida_env.aida_gym_env.AidaBulletEnv object at 0x102b10350>>
<bound method AidaBulletEnv._get_observation of <aida_env.aida_gym_env.AidaBulletEnv object at 0x102b10350>>
<bound method AidaBulletEnv._get_observation of <aida_env.aida_gym_env.AidaBulletEnv object at 0x102b10350>>
<bound method AidaBulletEnv._get_observation of <aida_env.aida_gym_env.AidaBulletEnv object at 0x102b10350>>
<bound method AidaBulletEnv._get_observation of <aida_env.aida_gym_env.AidaBulletEnv object at 0x102b10350>>
<bound method AidaBulletEnv._get_observation of <aida_env.aida_gym_env.AidaBulletEnv object at 0x102b10350>>
<bound method AidaB

KeyboardInterrupt: 

# Agents

An agent must contain a function named get_actions() (without any args, see example above) that will return the next set of actions to be performed on the environment. Make sure that this is respected so that the testing function work ( see below).
The test function will test how far the robot goes and return the distance from the start. This will be used to evaluate your solution.

In [ ]:
from aida_env.utils import test_agent_on_env

In [ ]:
test_agent_on_env(agent)